In [ ]:
import IPython
import numpy as np
import cv2
import pandas as pd
import os


from numpy import median

In [202]:
def listHistogramEuclidean(input_directory: str):
    histogramListsOfVideos = []
    # Get Histograms
    if os.path.isdir(input_directory):
        for filename in sorted(os.listdir(input_directory)):
            filepath = os.path.join(input_directory, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath) and extension.upper() in ['.MP4']: #just take these extensions
                cap = cv2.VideoCapture(filepath)
                histogramList = []
                while(cap.isOpened()):
                    ret, frame = cap.read()
                    if not ret:
                        cap.release()
                    if ret: 
                        hist = cv2.calcHist([frame],[2],None,[256],[0,256]) #tutorial #https://docs.opencv.org/4.x/d1/db7/tutorial_py_histogram_begins.html
                        histogramList.append(hist)
            histogramListsOfVideos.append(histogramList)
            
            
    # Get Euclidian Distances between consecutive frames
    euclDistListsOfVideos = []
    for histogramList in histogramListsOfVideos:
        euclDistList = [1]
        for i in np.arange(1, len(histogramList)):
            a = histogramList[i]
            b = histogramList[i-1]
            eucl = np.linalg.norm(a-b)
            euclDistList.append(eucl)
        euclDistListsOfVideos.append(euclDistList)
        #display(euclDistList)
    
    
    # Determine Cuts based on histogram eucl distances
    shotCutInFrameListsOfVideos = []
    for euclDistList in euclDistListsOfVideos:
        shotCutInFrameList = [1]
        medianDist = median(euclDistList)
        for i in np.arange(0, len(euclDistList)):
            if euclDistList[i] > medianDist * 6: # if eucl over 6 * the median, then its a cut
                shotCutInFrameList.append(i + 3) #because of Ground-Truth 3+ #offset
        shotCutInFrameListsOfVideos.append(shotCutInFrameList)
    
    
    return shotCutInFrameListsOfVideos
    

In [203]:
# https://docs.opencv.org/3.4/dd/d43/tutorial_py_video_display.html

def listHistogramIntersection(input_directory: str):
    histogramListsOfVideos = []
    # Get Histograms
    if os.path.isdir(input_directory):
        for filename in sorted(os.listdir(input_directory)):
            filepath = os.path.join(input_directory, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath) and extension.upper() in ['.MP4']: #just take these extensions
                cap = cv2.VideoCapture(filepath)
                histogramList = []
                while(cap.isOpened()):
                    ret, frame = cap.read()
                    if not ret:
                        cap.release()
                    if ret: 
                        hist = cv2.calcHist([frame],[2],None,[256],[0,256]) #tutorial #https://docs.opencv.org/4.x/d1/db7/tutorial_py_histogram_begins.html
                        histogramList.append(hist)
            histogramListsOfVideos.append(histogramList)
            
            
    # Get Intersections between consecutive frames
    intersectionListsOfVideos = []
    for histogramList in histogramListsOfVideos:
        intersectionList = []
        for i in np.arange(1, len(histogramList)):
            a = histogramList[i]
            b = histogramList[i-1]
            inters = cv2.compareHist(a, b, cv2.HISTCMP_INTERSECT)
            intersectionList.append(inters)
        intersectionListsOfVideos.append(intersectionList)
        #display(intersectionList)
    
    
    # Determine Cuts based on histogram intersection
    shotCutInFrameListsOfVideos = []
    for intersectionList in intersectionListsOfVideos:
        shotCutInFrameList = [1]
        medianInt = median(intersectionList)
        for i in np.arange(0, len(intersectionList)):
            if intersectionList[i] < medianInt * 0.9:
                shotCutInFrameList.append(i + 3) #because of Ground-Truth 3+ #offset
        shotCutInFrameListsOfVideos.append(shotCutInFrameList)
    
    
    return shotCutInFrameListsOfVideos
    

In [225]:
def listInterFrameDifference(input_directory: str):
    greyFrameListsOfVideos = []
    # Get Frames gray
    if os.path.isdir(input_directory):
        for filename in sorted(os.listdir(input_directory)):
            filepath = os.path.join(input_directory, filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath) and extension.upper() in ['.MP4']: #just take these extensions
                cap = cv2.VideoCapture(filepath)
                greyFrameList = []
                while(cap.isOpened()):
                    ret, frame = cap.read()
                    if not ret:
                        cap.release()
                    if ret: 
                        fr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                        
                        greyFrameList.append(fr)
            greyFrameListsOfVideos.append(greyFrameList)
            
            
    # Get frame difference between consecutive frames
    diffListsOfVideos = []
    for greyFrameList in greyFrameListsOfVideos:
        diffList = []
        for i in np.arange(1, len(greyFrameList)):
            a = greyFrameList[i]
            b = greyFrameList[i-1]
            frame_diff = sum(sum(cv2.absdiff(a,b)))
            diffList.append(frame_diff)
        diffListsOfVideos.append(diffList)
    
    
    # Determine Cuts based on frame difference 
    shotCutInFrameListsOfVideos = []
    for diffList in diffListsOfVideos:
        shotCutInFrameList = []
        medianDist = median(diffList)
        #display(medianDist)
        for i in np.arange(0, len(diffList)):
            if diffList[i] > medianDist * 1.01:
                shotCutInFrameList.append(i + 3) #because of Ground-Truth 3+ #offset
        shotCutInFrameListsOfVideos.append(shotCutInFrameList)
    
    
    return shotCutInFrameListsOfVideos
    

In [226]:
# fmeasure from previous assignment

def detect_and_compare_all(input_directory: str):
    dataFrame1 = pd.DataFrame(columns=['File', 'True', 'Missed', 'Recall', 'Precision', 'F1']) # Inter-Frame Distance
    dataFrame2 = pd.DataFrame(columns=['File', 'True', 'Missed','Recall', 'Precision', 'F1']) # Histogram intersection
    
    
    
    #read ground-trouth
    gtShotsList = []
    filenames = []
    if os.path.isdir("../gt_shots/TestSet/"):
        for filename in sorted(os.listdir("../gt_shots/TestSet/")):
            filenames.append(filename)
            cutList=[]
            filepath = os.path.join("../gt_shots/TestSet/", filename)
            name, extension = os.path.splitext(filename)
            if os.path.isfile(filepath) and extension.upper() in ['.TXT']: #just take these extensions
                with open(filepath, "r") as txt:
                    lines = txt.readlines()
                    for line in lines:
                        cutList.append(int(line.rstrip('\n')))
            gtShotsList.append(cutList)            
    
    
    
    
    histIntVideos = listHistogramIntersection(input_directory)
    frameDiffVideos = listInterFrameDifference(input_directory)
    
    # Histogram Intersection
    for i in range(0,len(gtShotsList)):
        histIntList = histIntVideos[i]
        gtList = gtShotsList[i]
        
        detected = list(set(gtList) & set(histIntList))
        missed = list(set(gtList) - set(histIntList))
        
        
        if len(gtList) > 0:
            recall = len(detected) / len(gtList)
        else:
            recall = 0
            
        if len(histIntList) > 0:
            precision = len(detected) / len(histIntList)
        else:
            precision = 0
        
        
        # f-measure
        beta = 0.5
        if precision > 0:
            fmeasure = ((pow(beta, 2)+1) * precision * recall) / (pow(beta, 2)*precision + recall)
        else:
            fmeasure = 0
        
        newRow = pd.DataFrame.from_records([{
                    'File': filenames[i],
                    'True': len(detected),
                    'Missed': len(missed),
                    'Recall': recall,
                    'Precision': precision,
                    'F1': fmeasure
                }])
        dataFrame1 = pd.concat([dataFrame1, newRow])
    
    
    # Inter-Frame Difference
    for i in range(0,len(gtShotsList)):
        frameDiffList = frameDiffVideos[i]
        gtList = gtShotsList[i]
        
        detected = list(set(gtList) & set(frameDiffList))
        missed = list(set(gtList) - set(frameDiffList))
        
        
        if len(gtList) > 0:
            recall = len(detected) / len(gtList)
        else:
            recall = 0
            
        if len(frameDiffList) > 0:
            precision = len(detected) / len(frameDiffList)
        else:
            precision = 0
        
        
        # f-measure
        beta = 0.5
        if precision > 0:
            fmeasure = ((pow(beta, 2)+1) * precision * recall) / (pow(beta, 2)*precision + recall)
        else:
            fmeasure = 0
        
        newRow = pd.DataFrame.from_records([{
                    'File': filenames[i],
                    'True': len(detected),
                    'Missed': len(missed),
                    'Recall': recall,
                    'Precision': precision,
                    'F1': fmeasure
                }])
        dataFrame2 = pd.concat([dataFrame2, newRow])
    
    
    
                
                
    dataFrame1 = dataFrame1.sort_values(by='File')
    dataFrame2 = dataFrame2.sort_values(by='File')
    
    return dataFrame1, dataFrame2

In [228]:
# histDiffDF, interFDDF = detect_and_compare_all("../videos/TestSet")

# display(histDiffDF)
# display(interFDDF)